In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import json
from PIL import Image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

data_dir = 'image_exp/Classification/DataFewShot/Train'
image_dataset =datasets.ImageFolder(data_dir,data_transform)

train_loader = torch.utils.data.DataLoader(image_dataset, batch_size=11, shuffle=True, num_workers=8)
class_names = image_dataset.classes
dataset_sizes = len(image_dataset)

In [ ]:
print(dataset_sizes)
for images, labels in train_loader:
    print(type(images),type(labels))
    print(images.size(),labels.size())
    print(labels)
    print(class_names)
    print(images[0])
    print('==================')
    images = images.view(11, -1)
    print(images)
    print(np.sum(images.numpy(), axis=1))

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out)

In [ ]:
class KNN(object):
    def __init__(self):
        pass

    def fit(self, train_imgs, labels):
        self.train_imgs = train_imgs.numpy()
        self.labels = labels.numpy()

    def predict(self, test_dataset):
        dis = self.compute_distances(test_dataset)
        return self.predict_labels(dis)

    def compute_distances(self, test_dataset):
        num_test = test_dataset.shape[0]
        num_train = self.train_imgs.shape[0]
        test_dataset = test_dataset.numpy()
        test_dataset = test_dataset.reshape(num_test, -1)
        #x2 = np.sum(test_dataset**2, axis=1).reshape((num_test, 1))
        #y2 = np.sum(self.train_imgs**2, axis=1).reshape((1, num_train))
        #xy = -2 * np.matmul(test_dataset, self.train_imgs.T)
        #dist = np.sqrt(x2 + xy + y2)
        dist = np.zeros((num_test, num_train))
        for i in range(num_test):
            for j in range(num_train):
                dist[i][j]=np.sum((self.train_imgs[j]-test_dataset[i])**2)
        return dist

    def predict_labels(self, dist, k=1):
        num_test = dist.shape[0]
        y_pred = np.zeros(num_test, dtype=int)
        for i in range(num_test):
            pred_x = np.argsort(dist[i])[:k]
            pred_y = [self.labels[val] for val in pred_x]
            y_pred[i] = pred_y[0]
        return y_pred


In [59]:
knn = KNN()
for images, labels in train_loader:
    images = images.view(11, -1)
    print(labels)
    knn.fit(images, labels)
    y_pred = knn.predict(images)
    print(y_pred)

tensor([ 7,  9, 10,  1,  4,  2,  6,  3,  0,  5,  8])
[ 7  9 10  1  4  2  6  3  0  5  8]
